## Item-to-Item

In [13]:
import sys
import pandas as pd
import scipy
import numpy as np
from scipy.sparse import csr_matrix

# эта функция нам поможет считывать данные
from os import path

data_dir = "data/ml-latest-small"
def read_csv(filename: str) -> pd.DataFrame:
    data = pd.read_csv(path.join(data_dir, filename + ".csv"))
    return data

ratings = read_csv("ratings")
movies = read_csv("movies")

In [2]:
# пользователи ставят фильмам оценки
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
# про фильмы знаем названия и жанры
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
print("максимальный ID фильма")
last_movie_id = ratings["movieId"].max()
print(last_movie_id)
print("всего фильмов")
print(len(ratings["movieId"].unique()))
print("максимальный ID пользователя")
last_user_id = ratings["userId"].max()
print(last_user_id)
print("всего пользователей")
print(len(ratings["userId"].unique()))

максимальный ID фильма
193609
всего фильмов
9724
максимальный ID пользователя
610
всего пользователей
610


In [5]:
ratings["movie_id"] = ratings["movieId"].astype("category").cat.codes.copy() + 1

In [7]:
print("максимальный ID фильма")
last_movie_id = ratings["movie_id"].max()
print(last_movie_id)
print("всего фильмов")
print(len(ratings["movie_id"].unique()))

максимальный ID фильма
9724
всего фильмов
9724


In [8]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [10]:
user_x_item = ratings[["userId", "movie_id"]].values
user_x_item

array([[   1,    1],
       [   1,    3],
       [   1,    6],
       ...,
       [ 610, 9445],
       [ 610, 9446],
       [ 610, 9486]], dtype=int64)

In [11]:
user_item_matrix = csr_matrix(
    (
        [1] * len(user_x_item),
        (
            [pair[0] for pair in user_x_item],
            [pair[1] for pair in user_x_item],
        )
    ),
    shape=(last_user_id + 1, last_movie_id + 1)
)

sparse_info(user_item_matrix)

Размерности матрицы: (611, 9725)
Ненулевых элементов в матрице: 100836
Доля ненулевых элементов: 0.01697011515531452
Среднее значение ненулевых элементов: 1.0
Максимальное значение ненулевых элементов: 1
Минимальное значение ненулевых элементов: 1


In [12]:
from scipy.spatial.distance import pdist, squareform

# pdist умеет всё "из коробки", но только с плотными матрицами
# плотная матрица может не умещаться в память - это нормально
similarity_matrix = squareform(
    1 - pdist(
        user_item_matrix.todense().T,
        metric="cosine"
    )
)
print(similarity_matrix.shape)

(9725, 9725)


In [13]:
print(movies[movies["title"].str.contains("Harry Potter")][["movieId", "title"]])
ratings[ratings["movieId"] == 4896].head(1)

      movieId                                              title
3574     4896  Harry Potter and the Sorcerer's Stone (a.k.a. ...
4076     5816     Harry Potter and the Chamber of Secrets (2002)
5166     8368    Harry Potter and the Prisoner of Azkaban (2004)
6062    40815         Harry Potter and the Goblet of Fire (2005)
6522    54001   Harry Potter and the Order of the Phoenix (2007)
7078    69844      Harry Potter and the Half-Blood Prince (2009)
7465    81834  Harry Potter and the Deathly Hallows: Part 1 (...
7644    88125  Harry Potter and the Deathly Hallows: Part 2 (...


,userId,movieId,rating,timestamp,movie_id
513,4,4896,4.0,1007574532,3570


In [14]:
# представление фильма в виде булевого вектора
movie_id = 3811
print(len(user_item_matrix[:, movie_id].data))
print(user_item_matrix[:, movie_id])

1
  (414, 0)	1


In [15]:
from scipy.spatial import distance

def get_similar_movies(movie_id: int, n: int) -> list:
    return (-similarity_matrix[movie_id, :]).argsort()[:n]

similar_movies = get_similar_movies(3811, 5)
movies[movies["movieId"].isin(
    ratings[ratings["movie_id"].isin(similar_movies)]["movieId"].tolist()
)]

,movieId,title,genres
3291,4451,Jump Tomorrow (2001),Comedy|Drama|Romance
3707,5112,Last Orders (2001),Drama
3860,5424,Harvard Man (2001),Crime|Drama|Romance|Thriller
5239,8587,Mayor of the Sunset Strip (2003),Documentary
9241,154358,The Barkley Marathons: The Race That Eats Its ...,Documentary


## Item-to-user

In [15]:
ratings = read_csv("ratings")
movies = read_csv("movies")
ratings["movie_id"] = ratings["movieId"].astype("category").cat.codes.copy() + 1

def sparse_info(sparse_matrix: csr_matrix):
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))
    
mean_rating = ratings["rating"].mean()
print("самое простое предсказание оценки - это средняя оценка")
print(mean_rating)

last_movie_id = ratings["movie_id"].max()
last_user_id = ratings["userId"].max()
user_x_item = ratings[["userId", "movie_id"]].values
# здесь уже на пересечении строк и столбцов матрицы стоят не единички
# а отклонение оценки от средней оценки
user_item_matrix = csr_matrix(
    (
        (ratings["rating"] - mean_rating).tolist(),
        (
            [pair[0] for pair in user_x_item],
            [pair[1] for pair in user_x_item],
        )
    ),
    shape=(last_user_id + 1, last_movie_id + 1),
    dtype=np.float32
)

sparse_info(user_item_matrix)

самое простое предсказание оценки - это средняя оценка
3.501556983616962
Размерности матрицы: (611, 9725)
Ненулевых элементов в матрице: 100836
Доля ненулевых элементов: 0.01697011515531452
Среднее значение ненулевых элементов: 1.4678314563809636e-08
Максимальное значение ненулевых элементов: 1.4984430074691772
Минимальное значение ненулевых элементов: -3.001556873321533


In [16]:
# теперь найдём матрицу схожести между фильмами
from sklearn.preprocessing import normalize, binarize

def get_cosine_similarity_matrix(user_item_matrix):
    # матрицы этого типа быстрее умножаются
    user_item_csr = user_item_matrix.tocsr()
    # нормализация и последующее умножение эквивалентно нахождению косинуса между столбцами матрицы
    user_item_normalized = normalize(user_item_csr, norm='l2', axis=0)
    return user_item_normalized.T.dot(user_item_normalized)

similarity_matrix = get_cosine_similarity_matrix(user_item_matrix)
sparse_info(similarity_matrix)

Размерности матрицы: (9725, 9725)
Ненулевых элементов в матрице: 26325068
Доля ненулевых элементов: 0.27834939499474626
Среднее значение ненулевых элементов: 0.05987884849309921
Максимальное значение ненулевых элементов: 1.0000015497207642
Минимальное значение ненулевых элементов: -1.0


In [17]:
from scipy.sparse import vstack, csr_matrix

# оставляем только top k схожих элементов (k ближайших соседей)
def get_top_k_in_a_row(similarity_matrix, row, k, no_diagonal):
    lil_row = similarity_matrix[row]
    # каждый фильм очень сильно схож сам с собой, но это бесполезная информация
    if no_diagonal:
        lil_row[0, row] = 0
    csr_row = lil_row.tocsr()
    csr_row.data[csr_row.data.argsort()[:-k]] = 0
    csr_row.eliminate_zeros()
    return csr_row

def get_top_k(similarity_matrix, k, no_diagonal=True):
    # с матрицами этого типа удобнее всего работать построчно
    lil_similarity = similarity_matrix.tolil()
    top_k_similarity_matrix = get_top_k_in_a_row(lil_similarity, 0, k, no_diagonal)
    for row in range(1, lil_similarity.shape[0]):
        if len(lil_similarity.rows[row]) > 0:
            csr_row = get_top_k_in_a_row(lil_similarity, row, k, no_diagonal)
            top_k_similarity_matrix = vstack([top_k_similarity_matrix, csr_row])
    return top_k_similarity_matrix

In [18]:
top_k_similarity_matrix = get_top_k(similarity_matrix, 5)
del similarity_matrix
print("ненулевых элементов стало гораздо меньше")
sparse_info(top_k_similarity_matrix)

ненулевых элементов стало гораздо меньше
Размерности матрицы: (9725, 9725)
Ненулевых элементов в матрице: 48620
Доля ненулевых элементов: 0.0005140859497359916
Среднее значение ненулевых элементов: 0.7763450741767883
Максимальное значение ненулевых элементов: 1.0000001192092896
Минимальное значение ненулевых элементов: -0.05091584473848343


In [19]:
# выделяем обучающую и тестовую выборки
from scipy.sparse import coo_matrix

np.random.seed(0)
train_percent = 0.1
user_item_matrix = user_item_matrix.tocoo()
train_split = np.random.choice(
    range(user_item_matrix.nnz),
    int(user_item_matrix.nnz * train_percent),
    replace=False
)
test_split = list(set(range(user_item_matrix.nnz)) - set(train_split))
train_matrix = coo_matrix(
    (
        user_item_matrix.data[train_split],
        (user_item_matrix.row[train_split], user_item_matrix.col[train_split])
    ),
    shape=user_item_matrix.shape
)
test_matrix = coo_matrix(
    (
        user_item_matrix.data[test_split],
        (user_item_matrix.row[test_split], user_item_matrix.col[test_split])
    ),
    shape=user_item_matrix.shape
)
print("Размер обучающей выборки:", train_matrix.nnz)
print("Размер тестовой выборки:", test_matrix.nnz)

Размер обучающей выборки: 10083
Размер тестовой выборки: 90753


In [20]:
#Среднеквадратичная ошибка (Root Mean Square Error, RMS Error, RMSE) - расстояние между двумя точками.
def RMSE(matrix1: coo_matrix, matrix2: coo_matrix):
    return np.sqrt(((matrix1 - matrix2).data ** 2).mean())

mean_rating_prediction = 0 * test_matrix
print("точность нашего baseline")
print(RMSE(mean_rating_prediction, test_matrix))

точность нашего baseline
1.0405655


In [21]:
# нахождение предсказаний эквивалентно умножению нормированной матрицы схожести на вектор оценок пользователя
normalized_similarity = normalize(top_k_similarity_matrix.tocsr(), norm="l1", axis=0)
raw_predictions = train_matrix.dot(normalized_similarity).tocoo()
print("получили предсказаний всего")
print(len(raw_predictions.data))

получили предсказаний всего
45383


In [22]:
filtered_predictions = binarize(test_matrix).multiply(raw_predictions)
print("оставили предсказаний для теста")
print(len(filtered_predictions.data))

оставили предсказаний для теста
5459


In [23]:
print(RMSE(filtered_predictions, test_matrix))

1.031941
